In [ ]:
import numpy as np
import jax.numpy as jnp
np.set_printoptions(formatter={'float': lambda x: "{0:0.5f}".format(x)})
import jax

In [ ]:
games = np.load("games.npz")
for game_name in sorted(games.keys()):
    G = games[game_name]
    P = G.shape[-1]
    A = G.shape[:-1]
    print(game_name + ":", P, "players,", A, "actions")

In [ ]:
chicken = games["chicken"]
pk3 = games["pk_3_actions"]
ttf = games["two_by_three_by_four"]
rps = games["rock_paper_scissors"]

To play nice with JAX, we need to represent a profiles as a 2D array instead of as a list of 1D arrays.
We accomplish this by padding out the actions dimension to the maximum number of actions available to any player, filling in zeros for the probability of invalid actions.

In [ ]:
def uniform_profile(game):
    num_players = game.shape[-1]
    max_actions = max(game.shape[:-1])
    profile = np.zeros([num_players, max_actions])
    for p in range(num_players):
        num_actions = game.shape[p]
        profile[p,:num_actions] = np.ones(num_actions) / num_actions
    return profile

def random_profile(game):
    num_players = game.shape[-1]
    max_actions = max(game.shape[:-1])
    profile = np.zeros([num_players, max_actions])
    for p in range(num_players):
        num_actions = game.shape[p]
        profile[p,:num_actions] = np.random.dirichlet([1]*num_actions)
    return profile

print(uniform_profile(ttf))

This requires a re-write of the `deviation_payoffs` function to slice out the appropriate sub-arrays.
Rather than making you do this yourself, I'm providing a working implementation that's vectorized and that should also play nice with JAX in other ways.

In [ ]:
def deviation_payoffs(game, profile, player):
    num_players = game.shape[-1]
    pay
    offs = game[...,player]
    for p in range(num_players):
        if p != player:
            a = game.shape[p]
            payoffs = jnp.swapaxes(jnp.swapaxes(payoffs, p, -1) * profile[p,:a], p, -1)
    payoffs = jnp.swapaxes(payoffs, player, -1)
    return jnp.apply_over_axes(jnp.sum, payoffs, range(num_players-1))

print(deviation_payoffs(chicken, uniform_profile(chicken), 0))

Helper functions for normalizing/projecting onto a probability simplex. Both functions take a vector and project it onto the probability simplex of the same dimension. `simplex_normalize` assumes all entries in the array are non-negative.

In [ ]:
def simplex_normalize(array):
    return array / np.sum(array)

_SIMPLEX_BIG = 1 / np.finfo(float).resolution
def simplex_project(array):
    """Return the projection onto the simplex"""
    array = np.asarray(array, float)
    # check(not np.isnan(array).any(), "can't project nan onto simplex: {}", array)
    # This fails for really large values, so we normalize the array so the
    # largest element has absolute value at most _SIMPLEX_BIG
    array = np.clip(array, -_SIMPLEX_BIG, _SIMPLEX_BIG)
    size = array.shape[-1]
    sort = -np.sort(-array, -1)
    rho = (1 - sort.cumsum(-1)) / np.arange(1, size + 1)
    inds = size - 1 - np.argmax((rho + sort > 0)[..., ::-1], -1)
    rho.shape = (-1, size)
    lam = rho[np.arange(rho.shape[0]), inds.flat]
    lam.shape = array.shape[:-1] + (1,)
    return np.maximum(array + lam, 0)

The `deviation_gains` function returns an array with the deviation gain for each of the player's actions; an efficient implementation should only call `deviation_payoffs` once, and should use vectorized operations to avoid loops. The `total_gain` function returns the sum of the the action-gains for each player & action; `deviation_gains` should be called once per player.

In [ ]:
def deviation_gains(game, profile, player):
    deviation = deviation_payoffs(game, profile, player)
    expected_utility = jnp.dot(deviation, profile[player])
    gain = jnp.maximum(0, deviation - expected_utility)
    return gain

def total_gain(game, profile):
    num_players = game.shape[-1]
    gain = 0
    for p in range(num_players):
        deviation = deviation_payoffs(game,profile,p)
        expected_util = jnp.dot(deviation,profile[p])
        for i in range(len(deviation[0])):
            gain += jnp.sum(jnp.maximum(0,deviation[i]-expected_util[0]))
    return gain

You should expand the following set of tests for the gain functions.

In [ ]:
def normalize(w):
    return w / jnp.sum(w, axis=1).reshape(w.shape[0],1)

def regret_matching(game, iterations=200, initial_profile=None, initial_weight=1):
    if initial_profile is None:
        initial_profile = uniform_profile(game)
    gains = [mixed_strat*initial_weight for mixed_strat in initial_profile]
    for i in range(iterations):
        for p in range(game.shape[-1]):
            deviation = deviation_gains(game, initial_profile, p)
            gains[p] = gains[p] + deviation 
        profile = [normalize(g) for g in gains]   
    return profile

In [ ]:
def simplotope_project(game, profile):
    projected = np.zeros_like(profile)
    for player in range(profile.shape[0]):
        num_Actions = game.shape[player]
        mix = profile[player, :num_Actions]
        projected[player, :num_Actions] = simplex_project(mix)
    return projected

def gradient_descent(game, iterations=200, initial_profile=None, step_size=0.01):
    num_players = game.shape[-1]
    if initial_profile is None:
        initial_profile = uniform_profile(game)
    gain_gradient = jax.grad(lambda prof: total_gain(game, prof))
    curr_profile = initial_profile
    for i in range(iterations):
        grad = gain_gradient(curr_profile)
        curr_profile = curr_profile - (step_size*grad)
        curr_profile = simplotope_project(game, curr_profile)
    return curr_profile

In [ ]:
def replicator_dynamics(game, iterations=200, initial_profile=None, min_payoffs=None):
    num_players = game.shape[-1]
    if initial_profile is None:
        initial_profile = uniform_profile(game)
    if min_payoffs is None:
        min_payoffs = game.min(axis=tuple(range(num_players)))
    
    curr_profile = initial_profile
    for i in range(iterations):
        new_profile = np.zeros_like(initial_profile)
        for p in range(game.shape[-1]):
            dev_pays = deviation_payoffs(game, curr_profile, p)
            dev_pays -= min_payoffs[p]
            new_profile[p] = dev_pays * curr_profile[p]
        curr_profile = normalize(new_profile)    
                
    return curr_profile

You have implemented the following helper functions on previous assignments; feel free to copy over existing code.

In [ ]:
def regret(game, profile, player):
    strat_util = 0
    utilities = deviation_payoffs(game, profile, player)
    utilities = utilities[0]
    highest = -float("inf")
    for a in range(len(profile[player])):
        strat_util += (utilities[a] * profile[player][a])
        if (utilities[a] > highest):
            highest = utilities[a]
    regret = highest - strat_util
    return regret


def is_epsilon_equilibrium(game, mixed_profile, epsilon):
    list_regret = []
    for p in range(len(mixed_profile)):
        list_regret.append(regret(game, mixed_profile, p))
    highest = max(list_regret)
    return epsilon >= highest

In [ ]:
def filter_regrets(game, candidate_equilibria, epsilon=1e-4):
    list_equilibria = []
    for p in range(len(candidate_equilibria)):
        if(is_epsilon_equilibrium(game, candidate_equilibria[p], epsilon)):
            list_equilibria.append(candidate_equilibria[p])
    return list_equilibria

def filter_unique(candidate_equilibria, min_dist=1e-2):
    size = len(candidate_equilibria)
    sorted_list = []
    unique_equilibria = []
    unique_equilibria = candidate_equilibria[0]
    for i in range(size, 1, 1):
        for u in range(len(unique_equilibria)):
            if(np.allclose(unique_equilibria[u], candidate_equilibria[i])):
                unique_equilibria.append(candidate_equilibria[i], atol = min_dist)
    return unique_equilibria


def Nash_local_search(game, method=gradient_descent, restarts=2, **search_kwds):
    candidate = []
    for i in range(restarts):
        prof = random_profile(game)
        candidate.append(method(game, initial_profile = prof, **search_kwds))
    print(candidate)
    candidate = filter_regrets(game, candidate)
    print(candidate)
    candidate = filter_unique(game, candidate)
    return candidate

print(Nash_local_search(chicken, method = gradient_descent, step_size =0.001, iterations = 200))


Test `Nash_local_search` with all three algorithms (RM, RD, GD) on several different games.